# RAG med Azure AI Agent Service och Semantic Kernel

Den här kodsnutten visar hur man skapar och hanterar en Azure AI-agent för retrieval-augmented generation (RAG) med hjälp av `Azure AI Agent Service` och `Semantic Kernel`. Agenten bearbetar användarfrågor baserat på den hämtade kontexten och ger korrekta svar därefter.


SQLite-versionfix  
Om du stöter på felet:  
```
RuntimeError: Your system has an unsupported version of sqlite3. Chroma requires sqlite3 >= 3.35.0
```  

Avkommentera denna kodblock i början av din notebook:  


In [ ]:
# %pip install pysqlite3-binary
# __import__('pysqlite3')
# import sys
# sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

### Importera paket
Följande kod importerar de nödvändiga paketen:


In [ ]:
# Azure imports for project client and credentials
from azure.ai.projects.models import FileSearchTool, OpenAIFile, VectorStore
from azure.identity.aio import DefaultAzureCredential

# Semantic Kernel imports
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentThread

# Retrieval-Augmented Generation med Semantic Kernel & Azure AI Agent Service

Det här exemplet visar hur man använder **Azure AI Agent Service** för att utföra **Retrieval-Augmented Generation (RAG)** genom att kombinera en språkmodell med domänspecifik kontext från ett uppladdat dokument.

### Så här fungerar det

1. **Dokumentuppladdning**: En markdown-fil (document.md) som innehåller information (Contosos reseförsäkringspolicy) laddas upp till agenttjänsten.

2. **Skapande av vektorlagring**: Dokumentet indexeras i en vektorlagring för att möjliggöra semantisk sökning över dess innehåll.

3. **Agentkonfiguration**: En agent skapas med modellen `gpt-4o` och följande strikta instruktioner:
   - Svara endast på frågor baserat på hämtat innehåll från dokumentet.
   - Avstå från att svara om frågan ligger utanför dokumentets omfattning.

4. **Integration av File Search Tool**: `FileSearchTool` registreras med agenten, vilket gör det möjligt för modellen att söka och hämta relevanta utdrag från det indexerade dokumentet under inferens.

5. **Användarinteraktion**: Användare kan ställa frågor. Om relevant information hittas i dokumentet genererar agenten ett välgrundat svar.  
   Om inte, svarar agenten uttryckligen att dokumentet inte innehåller tillräcklig information.


### Huvudfunktion



Se till att först köra `az login` med Azure CLI så att rätt autentiseringskontext tillhandahålls när du använder `DefaultAzureCredential`. Azure AI Agent Service använder inte API-nycklar.


In [ ]:
async def main():
    async with (
        DefaultAzureCredential() as creds,
        AzureAIAgent.create_client(credential=creds) as client,
    ):
        file: OpenAIFile = await client.agents.upload_file_and_poll(file_path="document.md", purpose="assistants")
        vector_store: VectorStore = await client.agents.create_vector_store_and_poll(
            file_ids=[file.id], name="my_vectorstore"
        )

        # Define agent name and instructions tailored for RAG.
        AGENT_NAME = "RAGAgent"
        AGENT_INSTRUCTIONS = """
        You are an AI assistant designed to answer user questions using only the information retrieved from the provided document(s).

        - If a user's question cannot be answered using the retrieved context, **you must clearly respond**: 
        "I'm sorry, but the uploaded document does not contain the necessary information to answer that question."
        - Do not answer from general knowledge or reasoning. Do not make assumptions or generate hypothetical explanations.
        - Do not provide definitions, tutorials, or commentary that is not explicitly grounded in the content of the uploaded file(s).
        - If a user asks a question like "What is a Neural Network?", and this is not discussed in the uploaded document, respond as instructed above.
        - For questions that do have relevant content in the document (e.g., Contoso's travel insurance coverage), respond accurately, and cite the document explicitly.

        You must behave as if you have no external knowledge beyond what is retrieved from the uploaded document.
        """

        
        # Create file search tool with uploaded resources
        file_search = FileSearchTool(vector_store_ids=[vector_store.id])

        # 3. Create an agent on the Azure AI agent service with the file search tool
        agent_definition = await client.agents.create_agent(
            model="gpt-4o",  # This model should match your Azure OpenAI deployment.
            name=AGENT_NAME,
            instructions=AGENT_INSTRUCTIONS,
            tools=file_search.definitions,
            tool_resources=file_search.resources,
        )
        
        # Create the Azure AI Agent using the client and definition.
        agent = AzureAIAgent(
            client=client,
            definition=agent_definition,
        )
        
        # Create a thread to hold the conversation
        # If no thread is provided, a new thread will be
        # created and returned with the initial response
        thread: AzureAIAgentThread | None = None
        
        # Example user queries.
        user_inputs = [
            "Can you explain Contoso's travel insurance coverage?",  # Relevant context.
            "What is a Neural Network?"  # No relevant context from the document. Will not contain a source annotation.
        ]
        
        try:
            for user_input in user_inputs:
                print(f"# User: '{user_input}'")
                # Invoke the agent for the specified thread for response
                async for response in agent.invoke(messages=user_input, thread=thread):
                    print(f"# {response.name}: {response}")
                    thread = response.thread
        finally:
            # Clean up resources.
            await thread.delete() if thread else None
            await client.agents.delete_vector_store(vector_store.id)
            await client.agents.delete_file(file.id)
            await client.agents.delete_agent(agent.id)
            print("\nCleaned up agent, thread, file, and vector store.")

await main()

Understood! Please provide the markdown file you'd like me to translate.



---

**Ansvarsfriskrivning**:  
Detta dokument har översatts med hjälp av AI-översättningstjänsten [Co-op Translator](https://github.com/Azure/co-op-translator). Även om vi strävar efter noggrannhet, bör du vara medveten om att automatiserade översättningar kan innehålla fel eller felaktigheter. Det ursprungliga dokumentet på dess originalspråk bör betraktas som den auktoritativa källan. För kritisk information rekommenderas professionell mänsklig översättning. Vi ansvarar inte för eventuella missförstånd eller feltolkningar som uppstår vid användning av denna översättning.
